# Shortest Path revisited and NP complete problems

## Bellman-Ford

### Single source shortest path

- Input: directed graph $G = (V, E)$, edge lengths $c_{e}$ for each $e \in E$, source vertex $s \in V$
- Goal: for every destonation $v \in V$ and source $s$ in $V$, compute the length (sum of edge costs) of the shortest $s-v$ path

### Dijkstra

- Good $O(mlogn)$ with heaps
- Not always correct with negative edge lenghts, not very distributed

### Negative cycles

- Compute the shortest $s-v$ path, with cycles allowed
    - Undefined (will keep traversing negative cycle)
- Compute shortest cycle-free $s-v$ path
    - NP-hard (no polynomial algorithm, unless P=NP)
- Assume input graph has no negative cycle
    - Then for every $v$, there is a shortest $s-v$ path with $\le n-1$ edges

### Optimal substructure

- Intuition: exploit sequential nature of paths, subpath of a shortest path should itself be shortest
- Key idea: artificially restrict the number of edges in path
- Lemma: lef $G=(V,E)$ be a directed graph with edge lengths $c_{e}$ and source vertex $s$ ($G$ might or might not have a negative cycle), for every $v \in V, i \in \{1,2 \dots \}$, et $P$ = shortest $s-v$ path with at most $i$ edges (cycles are permitted)
    - Case #1: if $P$ has $\le (i-1)$ edges, it is a shortest $s-v$ path with $\le (i-1)$ edges
    - Case #2: if $P$ has $i$ edges with last hop $(w,v)$, then $P^{'}$ is a shortest $s-v$ path with $\le (i-1)$ edges
- Proof
    - Case #1: (obvious) contradiction
    - Case #2: if $Q$ (from $s$ to $w$, $\le (i-1)$ edges) is shorter than $P^{'}$, then $Q + (w,v)(=P)$, which contradicts the optimality of $P$

### Recurrence

- Let $L_{i,v}$ = minimum length of a s-v path with $\le i$ edges 
    - With cycles allowed
    - Defined as $+\infty$ if no $s-v$ paths with $\le i$ edges 
- For every $v \in V$, $i = 1 \dots n$
    - $L_{i,v} = min\left[L_{i-1,v}, min\left[L_{i-1,v},c_{wv}\right]\right]$
- Correctness
    - How many candidates are there for an optimal solution to a subproblem involving the destination $v$? $1$+in-degree$(v)$
    - Brute-force search from the only ($1$+in-degree$(v)$) candidates (by the optimal substructure lemma)

### If no negative cycles
    - Shortest path do not have cycles (removing a cycle only decreases length)
    - Have $\le (n-1)$ edges
    - Point: if $G$ has no negative cycle, only need to solve subproblems up to $i = n-1$
    - Subproblems: compute $L_{i,v}$ for all $i = 1 \dots n-1$ and all $v$ in $V$
    
### Bellman-Ford algorithm

- Let $A$ = 2D array (index $i$ and $v$)
- Base case: $A[0,s] = 0$ and $A[0,v] = +\infty \ \forall v \ne s$
- For $i = 1 \dots n-1$
    - For each $v \in V$
        - $A[i,v] = min\left[A_{i-1,v}, min\left[A_{i-1,v},c_{wv}\right]\right]$
- If $G$ has no negative cycle, then answer is $A[n-1,v]$
- Runs in $O(mn)$

### Stopping early

- Suppose for some $j < n-1$, $A[j,v] = A[j-1,v]$ for all vertices $v$
    - For all $v$, all future $A[i,v]'s will be the same$
    - Can safely halt (since $A[j-1,v]$'s = correct shortest-path distances)
- Claim: $G$ has no negative-cost cycle (that is reachable from $s$) <=> in the extended Bellman-Ford algorithm, $A[n-1,v] = A[n,v] \ \forall v \in V$
- Consequence: can check for a negative cycle just by running Bellman-Ford for one extra iteration (run time still $O(mn)$)
- Proof
    - (=>) proved in correctness of Bellman-Ford
    - (<=) assume $A[n-1,v] = A[n,v] \ \forall v \in V$, assume also these are finite
    - Let $d(v)$ denote the common value of $A[n-1,v]$ and $A[n,v]$
    - Notice from algorithm: $d(v) \le d(w) + c_{wv} \ \forall$ edges $(w,v) \in E$
    - $\Sigma_{(w,v) \in c} \ge \Sigma_{(w,v) \in c}(d(w)-d(v)) = 0$

### Predecessor pointers

- Only need $A[i-1,v]$'s to compute $A[i,v]$'s
- Thus, only need $O(n)$ to remember current and last rounds of subproblems
- Compute a second table B where $B[i,v]$ = 2nd-to-last vertex on a shortest s-v path with $\le i $ edges (or NULL if no such path exists)
- Reconstruction: assume input graph $G$ has no negative cycles and we correctly compute $B[i,v]$'s
- Then, tracing back predecessor pointers $B[n-1,v]$'s from $v$ to $s$ yields a shortest $s-v$ path
- Base case: $B[0,v]$ = NULL $\forall v \in V$
- To compute $B[i,v]$ with $i > 0$
    - Case 1: $B[i,v] = B[i-1,v]$
    - Case 2: $B[i,v]$ = the vertex $x$, $w$ achieving the minimum (ex. the new last hop)
- Correctness: computation of $A[i,v]$ is brute-force search through the ($1$+in-degree$(v)$) possible optimal solutions, $B[i,v]$ is just cashing the last hop of the winner
- To reconstruct a negative-cost cycle, use depth-first search to check for a cycle of predessor pointers after each round (must be a negative cost cyle)

### Bellman-Ford to internet routing

- Switch from source-drive to destination driven (just reverse all directions in the Bellman-Ford algorithm)
    - Every vertex $v$ stores shortest-path distance from $v$ to destination $t$ and the first hop of a shortest-path (for all relevant destinations $t$)
    
### Handling asynchrony

- Can't assume all $A[i,v]$'s get computed before all $A[i-1,v]$'s
    - Fix: switch from "pull-based" to "push-based": as soon as $A[i,v] \lt A[i-1,v]$, $v$ notifies all of its neighbours
    - Algorithm guaranteed to converge eventually (assuming no negative cycles)
        - Updates strictly decrease sum of shortest-path estimates
        
### Handling failures

- Problem: convergence guaranteed only for static network (not true in practice)
    - Fix: each $V$ maintains entire shortest path to $t$, not just the next hop
        - Con: more space required
        - Pro: more robust to failures, permites more sophisticated route selection

## All pairs shortest path (APSP)

- Input: directed graph $G=(V,E)$ with edge cost $c_{e}$ for each edge $e \in E$ (no distinguished source vertex)
- Goal: compute the length of a shortest $u->v$ path for all pairs of vertices $u,v \in V$ or correctly that $G$ contains a negative cycle
- Dijkstra runs in $O(nmlogn)$
- Bellman-Ford runs in $O(n^{2}m)$

### Optimal substructure

- Key idea: order the vertices $V = {1,2 \dots n}$ arbitrarily, let $V^{k} = {1,2 \dots n}$
- Lemma: suppose $G$ has no negative cycle, fix source $i \in V$, destination $j \in V$, and $k \in {1,2 \dots n}$, let $P$ = shortest (cycle-free) $i-j$ path with all internal vertices $V^{(k-1)}$
    - Case #1: if $k$ not internal to $P$, then $P$ is a shortest $i-j$ (cycle-free) path with all internal vertices $V^{(k-1)}$
    - Case #2: if $k$ is internal to $P$, then
        - $P_{1}$ = shortest (cycle-free) $i-k$ path with all internal nodes in $V^{(k-1)}$
        - $P_{2}$ = shortest (cycle-free) $k-j$ path with all internal nodes in $V^{(k-1)}$

### Floyd-Warshall algorithm
    
- Let $A$ = 3D array (index $i,j,k$)
- $A[i,j,k]$ = length of a shortest i-j path with all internal nodes in $\{1 \dots k\}$
- Base case: $\forall i,j \in V$
    - $A[i,j,0]$ = 0 if $i$ = $j$, $C_{(ij)}$ if $(i,j)$ in $E$, $+\infty$ if $i$ != $j$ and $(i,j)$ not in $E$
- For $k =1 \dots n$
    - For $i = 1 \dots n$
        - For $j = 1 \dots n$
            $A[i,j,k]$ = $min\left[A[i,j,k-1], A[i,k,k-1] + A[k,j,k-1]\right]$
- Runs in $O(n^{3})$ ($O(1)$ per subproblem)
- What if input graph $G$ has a negative cycle? 
    - Will have $A[i,i,n] < 0$ for at least one $i \in V$ at the end of algorithm
- Reconstruct a shortest $i-j$ path? 
    - In addition to $A$, have Floyd-Warshall compute $B[i,j]$ = max level of an internal node on a shortest i-j path $\forall i,j \in V$ 
    - Reset $B[i,j] = k$ if 2nd case of recurrence used to compute $A[i,j,k]$
    - Can use $B[i,j]$'s to recursively reconstruct shortest paths!
    
### Rewritting technique

- APSP reduces to $n$ invocations of SSSP
    - Non-negative edge length: $O(mnlogn)$ via Dijkstra
    - General edge length: $O(mn^{2})$ via Bellman-Ford
- Johnson's algorithm: reduce all pairs shortest path tp 
    - n Dijkstra ($O(mnlogn)$)
    - 1 Bellman-Ford ($O(mn)$)
- Reweighting using vertex weights $\{p_{v}\}$ adds the same amount (namely, $p_{s} - p_{t}$) to every s-t path
- Reweighting always leaves the shortest path unchanged

### Reweighting 

- Define vertex weight $P_{v}$, which adds the same amount (namely, $P_{s}-P_{t}$ to every $s-t$ path)
- Reweighting always leaves the shortest path unchanged
- After reweighting, all edge length are non-negative

### Johnson's algorithm ($O(mnlogn)$)

1. Form $G^{'}$ by adding a new vertex $s$ and a new edge $(s,v)$ with length 0 for each $v$ in $G$
2. Run Bellman-Ford on $G^{'}$ with source vertex s
3. For each $v$ in $G$, define $p_{v}$ = length of a shortest s-v path in $G^{'}$. For each edge $e=(u,v)$ in $G$, define $c^{'}_{e}$ = $c_{e}$ + $p_{u}$ - $p_{v}$
4. For each vertex $u$ of $G$, run Dijkstra in $G$, with edge lengths $\{c^{'}_{e}\}$, with source vertex $u$, to compute the shortest path distance $d^{'}(u,v)$ for each $v$ in $G$
5. For each pair $u,v$ in $G$, return the shortest path distance $d(u,v)$ = $d^{'}(u,v)$ - $p_{u}$ + $p_{v}$

### Correctness

- Claim: for every edge $e=(u,v)$ of $G$, the reweighted length $c^{'}_{e}$ = $c_{e}$ + $p_{u}$ - $p_{v}$ is non-negative
- Proof: fix an edge $(u,v)$, by contradiction
     - $p_{u}$ = length of a shortest $s-u$ path of $G^{'}$
     - $p_{v}$ = length of a shortest $s-u$ path of $G^{'}$
     - Let $p$ = shortest $s-v$ path in $G^{'}$ (with length $p_{u}$ - exists by constructions of $G^{'}$)
         - $p+(u,v)$ = an $s-v$ path with length $p_{u} + c_{uv}$
         - Shortest $s-v$ path only shorter, so $p_{v} \le p_{u} + c_{uv}$
         - $c^{'}_{uv} = c_{uv} + p_{u} - p_{v} \ge 0$

In [ ]:
import numpy as np

In [ ]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path (string) -- location of file to read

    Returns:
    tuple_data (tuple) -- dictionary representing node & weight, integers reprsenting number of vertices and edges
    """

    data_dict = {}

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_vertices = int(data_array[0].split(" ")[0])
        num_edges = int(data_array[0].split(" ")[1])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            node1 = item.split(" ")[0]
            node2 = item.split(" ")[1]
            weight = int(item.split(" ")[2])
            data_dict[node1+"-"+node2] = weight
            
    tuple_data = (data_dict, num_vertices, num_edges)
    return tuple_data

In [ ]:
def all_pairs_shortest_path(data_dict, num_vertices):
    """
    Implement all pairs shortest path algorithm
    
    Args:
    data_dict (dictionary) -- stores node & weight
    num_vertices (integer) -- number of vertices in a graph
    
    Returns:
    smallest (string) -- returns the cost of shortest path (or "Negative cyle" if such exists) 
    """
    
    A = np.zeros((num_vertices, num_vertices, 2))
                  
    for i in range(0, num_vertices):
        for j in range(0, num_vertices):
            index = str(i+1) + "-" + str(j+1)
            if i == j:
                A[i][j][0] = 0
            if index in data_dict:
                A[i][j][0] = data_dict[index]
            if i != j and index not in data_dict:
                A[i][j][0] = 10000000


    smallest = 10000000      
    for k in range(1, num_vertices):
        for i in range(0, num_vertices):
            for j in range(0, num_vertices):
                A[i][j][1] = min(A[i][j][0], A[i][k][0] + A[k][j][0])
                if A[i][j][1] < smallest:
                    smallest = A[i][j][1]
                A[i][j][0] = A[i][j][1]
        print(k)
    
    for i in range(0, num_vertices):
        if A[i][i][1] < 0:
            smallest = "Negative cycle"
    
    if smallest == "Negative cycle":
        return "Negative cycle"
    else:
        return str(int(smallest))

In [ ]:
tuple_obj = open_file("data/all-pairs-shortest-path4.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "-41")

tuple_obj = open_file("data/all-pairs-shortest-path5.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "-89")

tuple_obj = open_file("data/all-pairs-shortest-path6.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "Negative cycle")

tuple_obj = open_file("data/all-pairs-shortest-path7.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "-2")

tuple_obj = open_file("data/all-pairs-shortest-path1.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "Negative cycle")

# tuple_obj = open_file("data/all-pairs-shortest-path2.txt")
# print(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]))

# tuple_obj = open_file("data/all-pairs-shortest-path3.txt")
# print(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]))

# negative
# -19

## NP complete

### Polynomial-time solvability

- There is an algorithm that correctly solves $O(n^{k})$ time
    - $k$ = a constant
    - $n$ = input length
- $P$ = a set of polynomial-time solvable problems
- Cycle-free shortest paths in graphs with negative cycles and knapsack are NP-complete problems

### Traveling salesman problem

- Input: complete undirected graph with non-negative edge costs
- Output: A min-cost tour (cycle that visits every vertex exactly once)
- Conjecture: there is no polynomial-time algorithm for TSP

### Reduction

- Problem $\sqcap_{1}$ reduces to problem $\sqcap_{2}$ if given a polynomial-time subroutine for $\sqcap_{2}$, can use it to solve $\sqcap_{1}$ in polynomial-time

### Completeness

- Suppose $\sqcap_{1}$ reduces to $\sqcap_{2}$
- Contrapositive: if $\sqcap_{1}$ is not in $P$, then neither is $\sqcap_{2}$
- That is, $\sqcap_{2}$ is at least as hard as $\sqcap_{1}$
- Definition: let $C$ = a set of problems. Problem $\sqcap$ is $C$-complete if
    - $\sqcap \in C$
    - Everything in $C$ reduces to $\sqcap$
- That is $\sqcap$ is the hardest problem in all of $C$

### NP-completeness

- A problem is in NP if
    - Solutions always have length polynomial in the input size
    - Solutions can be verified in polynomial time
- Every problem in NP can be solved by brute-force search in exponential time
- Vast majority of natural computational problems are in NP
- A polynomial-time algorithm for one NP-complete problem solves every problem in NP efficiently [P = NP]
- But generally, P != NP

### User guide on NP-completeness

- NP-completeness of $\pi$
    - Find a known NP-complte problem $\sqcap^{'}$
    - Prove that $\sqcap^{'}$ reduced to $\sqcap$
        - Implies that $\sqcap$ at least as hard as $\sqcap^{'}$
        - $\sqcap$ is NP-complete as well
                
1. Focus on computationally tractable special case
    - WIS in path graphs
    - Kanpsack with polynomial size capacity
    - 2 SAT instead of 3 SAT
    - Vertex cover when ORT is small
2. Heuristics
3. Solve in exponential time but faster than brute-force search
    - Knapsack ($O(n)$ instead of $2^{n}$)
    - TSP (~$2^{n}$ instead of ~$n!$)
    - Vertex cover ($2^{OPT}n$ instead of $n^{OPT}$)

### Vertex cover problem

- Given an undirected graph $G=(V,E)$
- Compute a minimum-cardinality subset $S \in V$ that contains at least one endpoint of each edge of $G$
- In general, NP-complete problem
- Given a positive integer $k$ as input, we want to check whether or not there is a vertex cover with size $\le k$
- Could try all possibilities, would take $\theta{n^{k}}$ time

### Substructure lemma

- Consider graph $G$, edge $(u,v) \in G$, integer $k \ge 1$
- Let $G_{u} = G$ with $u$ and its incident edged deleted
- Let $G_{v} = G$ with $v$ and its incident edged deleted
- Then, $G$ has a vertex cover of size $k$ <=> $G_{u}$ or $G_{v}$ or both have a vertex cover of size $(k-1)$
- Proof
    - (<=) suppose $G_{u}$ (say) has a vertex cover $s$ of size $k-1$, write $E = E_{u}$ (inside $G_{u}$) $\cup$ $F_{u}$ (incident to $u$) since $s$ has an endpoint of each edge of $E_{u}, S \cup \{u\}$ is a vertex cover (of size $k$) of $G$
    - (=>) let $s$ = a vertex cover of $G$ of size $k$, since $(u,v)$ an edge of $G$, at least one of $u,v$ (say $u$) is in $S$, since no edges of $E_{u}$ incident of $u, s-\{u\}$ must be a vertex cover (of size $k-1$) of $G_{u}$

### Recurrence

- Ignore base case
- Pick an arbitrary edge $(u,v)$ in $E$
- Recursively search for a vertex cover $S$ of size $(k-1)$ in $G_{u}$. If found, return $S$ plus $u$
- Recursively search for a vertex cover $S$ of size $(k-1)$ in $G_{v}$. If found, return $S$ plus $v$

### Traveling salesman problem

- Given undirected graph with non-negative edge cost, find min cost to visit all vertices
- To enforce constraint that each vertex visited exactly once, need to remember the "identities" of vertices visited in a sub-problem
- Sub-problem: for every destination $j = \{1 \dots n \}$, every subset $S$ in $\{1 \dots n \}$ that contains $1$ and $j$
    - Let $L_{s,j}$ = minimum length of a path from $1$ to $j$ that visits precisely the vertices of $S$ (exactly once each)
- Let $p$ be a shortest path from $1$ to $j$ that visits the vertices $S$ (exactly once each) if last hop of $p$ is $(k,j)$, then $p^{'}$ is a shortest path from $1$ to $k$ that visits every vertex of $S - \{j\}$ exactly once

Recurrence
- $L_{s,j} = min\left[L_{s-\{j\},k} + C_{kj}\right]$ ($k$ in $S$, $k != j$)

### Algorithm

- Let $A$ = 2D array, indexed by subsets $S$ in $\{1 \dots n\}$ that contain $1$ and destinations $j$ in $\{1 \dots n\}$
- Base case: $A[s,1]$ = $0$ if $S = \{1\}$ , $+\infty$ otherwise
- For $m = 2,3,4 \dots n$ [$m$ = sub-problem size]
    - For each set $S \in \{1 \dots n\}$ of size $m$ that contains $1$
        - For each $j \in S, j=1$
            - $A_{s,j} = min\left[A_{s-\{j\},k} + C_{kj}\right]$ ($k$ in $S$, $k != j$)
- Return $min\left[A[\{1,2,3,...,n\}, j] + C_{j1}\right]$
- Runs in $O(n2^{n})O(n) = O(n^{2}2^{n})$

In [ ]:
from queue import *
from math import sqrt
import numpy as np

In [ ]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path (string) -- location of file to read

    Returns:
    tuple_data (tuple) -- dictionary storing coordinates and integer reprsenting number of cities in total
    """

    data_dict = {}
    index = 0

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_cities = int(data_array[0])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            x = float(item.split(" ")[0])
            y = float(item.split(" ")[1])
            data_dict[index] = (x,y)
            index += 1
            
    tuple_data = (data_dict, num_cities)
    return tuple_data

In [ ]:
def count_ones(binary):
    """
    Count number of '1' in the binary string
    
    Args:
    binary (string) -- a binary number
    
    Returns:
    count (integer) -- number of '1' in the binary string
    """
    
    count = 0
    for char in binary:
        if char == "1":
            count += 1
            
    return count

In [ ]:
def find_ones(m, S):
    """
    Search through binary string in a given list and find binary string whose count of '1's matches the given number  
    
    Args:
    m (integer) -- a number 
    S (list) -- binary strings
    
    Returns
    ret (list) -- binarys string whose count of '1's matches the given number
    """
    
    ret = []
    for binary in S:
        if count_ones(binary) == m:
            ret.append(binary)
            
    return ret

In [ ]:
def generate_binary(n, S, max_len):
    """
    Generate binary string
    
    Args:
    n (integer) -- number of binary strings to generate
    S (list) -- stores binary strings
    max_len (integer) -- length of each binary string
    
    Returns:
    None
    """

    # Create an empty queue
    q = Queue()

    # Enqueu the first binary number
    q.put("1")

    # This loop is like BFS of a tree with 1 as root
    # 0 as left child and 1 as right child and so on
    while(n>0):
        n-= 1
        # Print the front of queue
        s1 = q.get()

        zeros = ""
        num_zeros_to_append = max_len - len(s1)
        for i in range(0, num_zeros_to_append):
            zeros += "0"
        S.append(zeros+s1)

        s2 = s1 # Store s1 before changing it

        # Append "0" to s1 and enqueue it
        q.put(s1+"0")

        # Append "1" to s2 and enqueue it. Note that s2
        # contains the previous front
        q.put(s2+"1")

In [ ]:
def get_euclidean_distance(a, b, data_dict):
    """
    Find Euclidean distance between two 2-dimensional coordinate
    
    Args:
    a (integer) - an index to locate tuple representing the first coordinate
    b (integer) - an index to locate tuple representing the second coordinate
    data_dict (dictionary) -- stores coordinates
    
    Returns:
    dist (integer) -- Euclidean distance between two coordinates
    """
    
    x1 = data_dict[a][0]
    y1 = data_dict[a][1]
    x2 = data_dict[b][0]
    y2 = data_dict[b][1]

    dist = sqrt((x1-x2)**2 + (y1-y2)**2)
    return dist

In [ ]:
def convert_bit(j, binary):
    """
    Convert a specific bit in a binary string from 1 to 0
    
    Args:
    j (integer) -- an index of binary to convert
    binary (string) -- represets the binary before conversion
    
    Returns:
    bit (string) -- represets the binary after conversion
    """
    
    bit = ""
    for i in range(0, len(binary)):
        if i == j and binary[i] == "1":
            bit += "0"
        else:
            bit += binary[i]
            
    return bit

In [ ]:
def find_minimum(j, A, D, binary, num_cities):
    """
    Find the minimum of sum of specific values from dictionary A and 2-dimensional array D
    
    Args:
    j (integer) -- a particular index to convert bit
    A (dictionary) -- stores a binary string as key and an array as value
    D (list of list) -- 2-dimensional array that holds Euclidean distances between all coordinates
    binary (string) -- a binary string
    num_cities (integer) -- the number of cities to vist (size n of the problem)
    
    Returns:
    smallest (integer) -- minimun cost of the trip
    """
    
    smallest = 1000000
    for k in range(0, num_cities):
        if k != j and binary[k] == "1":
            new_binary = convert_bit(j, binary)
            candidate = A[new_binary][k] + D[j][k]
            if candidate < smallest:
                smallest = candidate

    return smallest

In [ ]:
def travelling_salesman_problem(data_dict, num_cities):
    """
    Implement dynamic programming algorithm to solve travelling salesman problem
    
    Args:
    data_dict (dictionary ) --  storing coordinates 
    num_cities (integer) -- the number of cities in total
    
    Returns:
    smallest (integer) -- minimun cost of the trip
    """
    
    # Construct binary string where 1 means the node presented by that index exists in the set, 0 otherwise
    num_cases_of_set = 2**(num_cities-1)
    S = []
    generate_binary(num_cases_of_set, S, num_cities-1)

    # The set S must contain the fist node, this first digit of binary must always be 1
    del S[len(S)-1]
    zeros = ""
    for i in range(0, num_cities-1):
        zeros += "0"
    S.insert(0, zeros)

    for i in range(0, len(S)):
        S[i] = "1" + S[i]

    # print("S")
    # print(S)

    # Initialize dictionary A
    A = {}
    for binary in S:
        A[binary] = []
        if binary[0] == "1" and count_ones(binary) == 1:
            A[binary].append(0)
        else:
            A[binary].append(10000000)
        for i in range(1, num_cities):
            A[binary].append(10000000)

    # print("A")
    # print(A)

    # Pre-calculate euclidean distances
    D = np.zeros((num_cities, num_cities))
    for i in range(0, num_cities):
        for j in range(0, num_cities):
            D[i][j] = get_euclidean_distance(i, j, data_dict)


    for m in range(2, num_cities+1):
        # for each set in S with m number of 1's
        for binary in find_ones(m, S):
            # for 1's in S that is not the first 1
            for j in range(1, len(binary)):
                if binary[j] == "1":
                    A[binary][j] = find_minimum(j, A, D, binary, num_cities)

    # print("final A")
    # print(A)

    all_in_set = ""
    smallest = 10000000
    for char in range(0, num_cities):
        all_in_set += "1"
    for j in range(0, num_cities):
        candidate = A[all_in_set][j] + get_euclidean_distance(j, 0, data_dict)
        if candidate < smallest:
            smallest = candidate

    return int(np.floor(smallest))

In [ ]:
tuple_obj = open_file("data/traveling-salesman-problem-test1.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 7)

tuple_obj = open_file("data/traveling-salesman-problem-test2.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 10)

tuple_obj = open_file("data/traveling-salesman-problem-test3.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 12)

tuple_obj = open_file("data/traveling-salesman-problem-test4.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 14)

tuple_obj = open_file("data/traveling-salesman-problem.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 26442)

## Approximation algorithm for NP complte problem (heuristics)

### Greedy knapsack heuristic

- Ideally items with large value, but small size
- Step1: sort adn reindex items so that
    - $\dfrac{v_{1}}{w_{1}} \ge \dfrac{v_{2}}{w_{2}} \ge \dots \ge \dfrac{v_{n}}{w_{n}}$
- Step2: pack items in this order until one doesn't fit , then halt
- Step3: return either solution from above, or the maximun valuable item, whichever is better
- Value of 3-step greedy solution is always $\le 50%$ value of an optimal solution (also, runs $O(nlogn)$ time)

### Greedy fraction solution

- What if we were allowed to fill fully the knapsack using a suitable "fraction" (like 70%) of item $(k+1)$? (value of which is "pro-rated")
- Claim: greedy fractional solution is at least as good as every non-fractional feasible solution
- Proof:
    - (1) Let $S$ = an arbitrary feasible solution
    - (2) Suppose $l$ units of knapsack filled by $S$ with items not packed by the greedy fractional solution
    - (3) Must be at least $l$ units of knapsack filled by greedy fractional solution not packed by $S$
    - (4) By greedy criterion, items in (3) have larger $v_{i}/w_{i}$ than those (2) (more valuable use of space)
    - (5) Total value of greedy fractional solution at least that of $S$

### Arbitrarily good approximation

- Goal: for a user-specified parameter $\epsilon \gt 0$, guarantee a $(1-\epsilon)$-approximation
- Run time will increase as $\epsilon$ decreases

### Rounding item values

- Exactly solve a slightly incorrect, but easier, kanpsack instance
- If the $w_{i}'s$ and $w$ are integers, can solve the knapsack problem via dynamic programming in $O(nW)$ time
- Or if $v_{i}$'s are integers, can solve knapsack via dynamic programming in $O(n^{2}v_{max})$ time, where $v_{max} = max_{i}\{v_{i}\}$
- If all $v_{i}$'s are small integers (polynomial in $n$) then we already know a poly-time algorithm
- Plan: throw out lower-order bits of the $v_{i}$'s!

### Dynamic programming heuristic

- Step #1
    - Round each $v_{i}$ down to the nearest multiple of $m$ (where $m$ depends on $\epsilon$)
    - Larger $m$ => throw out more info => less accuracy (where $m$ depends on $\epsilon$)
    - Divide the result by $m$ to get $\hat{v_{i}}$'s integers ($\hat{v_{i}}$ = floor($\dfrac{v_{i}}{m}$))
    
- Step #2
    - Solve using dynamic programming with sizes $\hat{v_{1}} \dots \hat{v_{n}}$, weights $w_{1} \dots w_{n}$, and maximun weight limit $W$
    - Runs in $O(n^{2}max_{i}\hat{v_{i}})$

### Sub-problem

- For $i = 0 \dots n$ and $x = 0 \dots nv_{max}$, define $S_{i,x}$ = minimum total size needed to achieve value $\ge x$ while using only the first $i$ items (or $+\infty$ if impossible)

### Recurrence

- $S_{i,x} = min\left[S_{(i-1),x}, w_{i} + S_{(i-1),(x-v_{i})}\right]$ where $S_{(i-1),(x-v_{i})} = 0$ if $v_{i} \ge x$

### Algorithm

- Let $A$ = 2D array ($i$ = $0$ to $n$, $x$ = $0$ to $nv_{max}$)
- Base case: $A[0,x]$ = $0$ if $x = 0$, $+\infty$ otherwise
- For $i = 1 \dots n$
    - For $x = 0 \dots nv_{max}$
        - $A[i,x]$ = $min\left[A[i-1,x], w_{i} + A[i-1,x-v_{i}]\right]$ where $A[i-1,x-v_{i}] = 0$ if $v_{i} \ge x$
- Return the largest $x$ such that $A[n,x] \le W$
- Runs in $O(n^{2}v_{max})$

### Analysis

- Figure out how big we can take $m$, subject to achieving a $(1-\epsilon)$-approximation
- Plug in this value of $m$ to determine running time
- Since we rounded down to the nearest multiple of $m, m\hat{v_{i}} \in [v_{i}-m, v_{i}]$ for each item $i$
- Thus, $v_{i} \le m\hat{v_{i}}$, $m\hat{v_{i}} \ge v_{i}-m$
- Also, if $S^{*}$ = optimal solution to the original problem (with the original $v_{i}$'s), and $S$ = our heuristic's solution, then

$\displaystyle\sum_{i \in S}\hat{v_{i}} \ge \displaystyle\sum_{i \in S^{*}}\hat{v_{i}}$ (since $S$ is optimal for the $\hat{v_{i}}$'s)

- $S$ = our solution, $S^{*}$ = optimal solution

$\displaystyle\sum_{i \in S} \ge m\displaystyle\sum_{i \in S}\hat{v_{i}} \ge m\displaystyle\sum_{i \in S^{*}}\hat{v_{i}} \ge \displaystyle\sum_{i \in S^{*}}(v_{i}m)$

- Thus, 

$\displaystyle\sum_{i \in S}v_{i} \ge \left(\displaystyle\sum_{i \in S^{*}}v_{i}\right) - mn$

- Contraint,

$\displaystyle\sum_{i \in S}v_{i} \ge (1-\epsilon)\displaystyle\sum_{i \in S^{*}}v_{i}$

- Choose $m$ small enough that $mn \le \epsilon\displaystyle\sum_{i \in S^{*}}v_{i}$
- Set $m$ so that $mn = \epsilon v_{max}$
- Setting $m = \dfrac{\epsilon v_{max}}{n}$ guarantees that value of our solution is $\ge (1-\epsilon)$(value of optimal solution)
- Recall runtime: $O(n^{2}\hat{v}_{max})$
- For every item $i, \hat{v}_{i} \le \dfrac{v_{i}}{m} \le \dfrac{v_{max}}{m} = v_{max}\dfrac{n}{\epsilon v_{max}} = \dfrac{n}{\epsilon}$
- Runs in: $O(n^{3}/\epsilon)$

In [ ]:
from math import sqrt

In [ ]:
def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path (string) -- location of file to read

    Returns:
    tuple_data (tuple) -- dictionary storing coordinates and integer reprsenting number of cities in total
    """

    data_dict = {}
    
    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_cities = int(data_array[0])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            index = int(item.split(" ")[0])
            x = float(item.split(" ")[1])
            y = float(item.split(" ")[2])
            data_dict[index] = (x,y)
            index += 1
            
    tuple_data = (data_dict, num_cities)
    return tuple_data

In [ ]:
def find_closest_city(city, data_dict, S):
    """
    Find a neighbouring city that has the minimum distance from the current city
    
    Args:
    city (integer) -- represents a city
    data_dict (dictionary) -- stores (x,y) coordinates of each city
    S (dictionary) -- stores all visited cities
    
    Returns:
    tuple_data (tuple) -- integer (city with minimum distance) and integer (minimum distance)
    """
    
    city_with_minimum_dist = 0
    minimum_dist = 10000000
    
    x1 = data_dict[city][0]
    y1 = data_dict[city][1]
    
    for key in data_dict:
        if key not in S:
            x2 = data_dict[key][0]
            y2 = data_dict[key][1]

            dist = sqrt((x1-x2)**2 + (y1-y2)**2)
            if dist < minimum_dist:
                minimum_dist = dist
                city_with_minimum_dist = key
            elif dist == minimum_dist:
                if city_with_minimum_dist > key:
                    city_with_minimum_dist = key
        
    tuple_data = (city_with_minimum_dist, minimum_dist)
    return tuple_data

In [ ]:
def traveling_salesman_problem_heuristic(data_dict, num_cities):
    """
    Implement heuristic algorithm for traveling salesman problem
    
    Args:
    data_dict (dictionary) -- stores (x,y) coordinates of each city
    num_cities (integer) -- the number of cities in total
    
    Returns:
    ret_val (integer) -- the totla cost of trip
    """
    
    S = {}
    D = []
    S[1] = "visited" # start tour at the first city
    num_visited_cities = 1
    current_city = 1

    while num_visited_cities < num_cities:
        result = find_closest_city(current_city, data_dict, S)
        S[result[0]] = "visited"
        current_city = result[0]
        D.append(result[1])
        num_visited_cities += 1

    x1 = data_dict[current_city][0]
    y1 = data_dict[current_city][1]
    x2 = data_dict[1][0]
    y2 = data_dict[1][1]
    final_dist = sqrt((x1-x2)**2 + (y1-y2)**2)
    
    ret_val = sum(D)+final_dist
    ret_val = int(ret_val)
    return ret_val

In [ ]:
tuple_obj = open_file("data/traveling-salesman-problem-heuristic-test1.txt")
assert(traveling_salesman_problem_heuristic(tuple_obj[0], tuple_obj[1]) == 15)

tuple_obj = open_file("data/traveling-salesman-problem-heuristic-test2.txt")
assert(traveling_salesman_problem_heuristic(tuple_obj[0], tuple_obj[1]) == 23)

tuple_obj = open_file("data/traveling-salesman-problem-heuristic-test3.txt")
assert(traveling_salesman_problem_heuristic(tuple_obj[0], tuple_obj[1]) == 716)

tuple_obj = open_file("data/traveling-salesman-problem-heuristic.txt")
assert(traveling_salesman_problem_heuristic(tuple_obj[0], tuple_obj[1]) == 1203406)

## Local search

### Maximum cut problem

- Input: an undirected grapg $G = (V,E)$
- Goal: a cut $(A,B)$ - a parition of $V$ into two non-empty sets - that maximizes the number of crossing edges
- NP-complete
- Computationally tractable special case: Bipartite graphs (where there is a cut such that all edges are crossing)

### Local search algorithm

- For a cut $(A,B)$ and a vertex $V$, define
    - $c_{v}(A,B)$ = number of edges incident on $v$ that corss $(A,B)$
    - $d_{v}(A,B)$ = number of edges incident on $v$ that don't corss $(A,B)$
- Let $(A,B)$ be an arbitrary cut of $G$
- While there is a vertex $v$ with $d_{v}(A,B) \gt c_{v}(A,B)$
    - Move $v$ to other side of the cut (increase number of crossing edges by $d_{v}(A,B) - c_{v}(A,B) \gt 0$)
- Return final cut $(A,B)$
- Terminates within $n\choose{2}$ iterations

### Performance guarantee

- Theorem: local search algorithm always outputs a cut in which the number of crossing edges is at least 50% of the maximum possible (even 50% of $|E|$)
- Expected number of crossing edges of a random cut already is $\dfrac{1}{2}|E|$
    - Consider a random cut$(A,B)$, for edge $e \in E$, define 
        - $X_{e} = 1$ if $e$ crosses$(A,B)$, 0 otherwise
    - We have $E[X_{e}] = Pr[X_{e} = 1] = \dfrac{1}{2}$
    - $E[$# crossing edges$] = E\left[\displaystyle\sum_{e}X_{e}\right] = \displaystyle\sum_{e}E[X_{e}] = \dfrac{|E|}{2}$ 
- let $(A,B)$ be a locally optimal cut, then for every vertex $v, d_{v}(A,B) \le c_{v}(A,B)$, summing over all $v \in V$
    - $\displaystyle\sum_{v \in V}d_{v}(A,B) \le  \displaystyle\sum_{v \in V}c_{v}(A,B)$
- so, $2$[# of non-crossing edges] $\le 2$[# of crossing edges] => $2|E| \le 4$[# crossing edges] => # of crossing edgs $\le \dfrac{1}{2}|E|$

### Weighted maximum cut problem

- Each edge $e \in E$ has a non-negative weight $w_{e}$, want to maximize total weight of crossing edges
- (1) Local search still well defined
- (2) Performance guarantee of 50% still holds for locally optimal cuts
- (3) No longer guaranteed to converge in polynomial time

### Principles in local search 

- Let $x$ = set of candidate solution to a problem
- Ex. cuts of a graph TSP tours, CSP variable assignment

Neighbourhoods
- Let $x$ = set of candidates solutions to a problem
- For each $x \in X$, specify which $y \in X$ are its neighbours
- $x,y$ are neighbouring cuts <=> differ by moving one vertex
- $x,y$ are neighbouring variable assignments <=> differ in the value of a single variable
- $x,y$ are neighbouring TSP tours <=> differ by 2 edges

## Generic local search algorithm

- Let $x$ = some initial solution
- While the current solution $x$ has a superior neighbouring solution $y$, set $x = y$
- Return the final (locally optimal) solution $x$

FAQ
- How to pick initial soltion $x$?
    - Random
        - Run many independent trials of local search, return the best locally optimal solution found
    - Best heuristic
        - Use local search as a postprocessing step to make your solution even better
- If there are several superior neighbouring $y$, which to choose? 
    - Random
    - Biggest improvement
    - More complex heuristics
- How to define neighbourhoods?
    - Better neighbourhoods => slower to verify local optimality, but fewer (bad) local optima
    - Find "sweat spots" between solution quality and efficient searchability
- Is local search guaranteed to terminate?
    - If $x$ is finite and every local step improves some objective function, then yes
- Is local search guaranteed to converge quickly? 
    - Usually not
- Are locally optimal solutions generally good approximations to globally optimal ones? 
    - No (but you can run many times and pick the best)

### 2-SAT

- Input: 
    - (1) $n$ Boolean variables $x_{1} \dots x_{n}$ (True or False) 
    - (2) $m$ clauses of 2 literal each ($x_{i}$ or $\neg x_{i}$)
- Output: "yes" if there is an assignment that simultaneously satisfies every class. "no" otherwise
- Ex. "yes" when $x_{1} = x_{3}$ = TRUE and $x_{2} = x_{4}$ = FALSE 
- Can be solved in polynomial time!
    - Reduction to computing strongly connected components
    - "Backtracking" works in polynomial time
    - Randomized local search

### 3-SAT

- NP complete
    - Brute-force search $~ 2^{n} time$
    - Can get time $~\left(\dfrac{4}{3}\right)^{n}$ via randomized local search

### Papadimitriou's 2-SAT algorithm

- Repeat $log_{2}n$ times (n = number of variables)
    - Choose random initial assignment
    - Repeat $2n^{2}$ times
        - If current assignment satisfies all clauses, halt and report this
        - Else, pick arbitrary unsatisfied clause and flip the value of one of its variables (choose between the two unformly at random)
    - Report "unsatisfiable"

Advantages
- Runs in polynomial time
- Always correct on unsatisfiable instances

### Random walks

- Initiall (at time $0$), at position $0$
- At each time step, your position goes up or down by $1$, with 50/50 chance (except if at position $0$, in which case you move to position $1$ with 100 chance)
- Let $Z_{i}$ = number of random walk steps to get to $n$ from $i (Z_{0} = T_{n})$
- Edge cases: $E[Z_{n}] = 0, E[Z_{0}] = 1 + E[Z_{1}]$
- For $i \in \{1,2 \dots n-1\}$
    - $E[Z_{i}] = Pr[$go left$]E[Z_{i}|$go left$] + Pr[$go right$]E[Z_{i}|$go right$] = 1 + \dfrac{1}{2}E[Z_{i+1}] + \dfrac{1}{2}E[Z_{i-1}]$
    - Re-arranging: $E[Z_{i}] - E[Z_{i+1}] = E[Z_{i-1}] - E[Z_{i}] + 2$
    - $(E[Z_{0}]-E[Z_{1}]) + (E[Z_{1}]-E[Z_{2}]) + \dots + (E[Z_{n-1}]-E[Z_{n}]) (\dfrac{n}{2}$ pairs of numbers, each sums to $2n) => E[Z_{0}] = n^{2} = E{T_{n}}$
- For an integer $n \ge 0$, let $T_{n}$ = number of steps until random walk reaches position $n$, then $E[T_{n}] = \theta{(n^{2})}$
- Corollary: $Pr[T_{n} > 2n^{2}] \ge \dfrac{1}{2}$ (special case of Markov's inequality)
- Proof
    - Let $p$ denote $Pr[T_{n} > 2n^{2}]$
    - We have $n^{2} = E[T_{n}] = \displaystyle\sum_{k=0}^{2n^{2}}kPr[T_{n}=k] + \displaystyle\sum_{k=2n^{2}+1}^{\infty}kPr[T_{n}=k] \ge 2n^{2}Pr[T_{n} \ge 2n^{2}] = 2n^{2}p => p \le \dfrac{1}{2}$
    
### Satisfiable instances

- Theorem: for a satisfiable 2-SAT instance with $n$ variables, Papadimitriou's algorithm produces a satisfying assignment with probability $\ge 1-\dfrac{1}{n}$
- Proof
    - Fix an arbitrary satisfying assignment $a^{*}$
    - Let $a_{t}$ = algorithm's assignment after inner iteration $t (t = 0,1 \dots 2n^{2})$ (a random variable)
    - Let $X_{t}$ = number of variables on which $a_{t}$ and $a^{*}$ agree $X_{t} \in \{0,1 \dots n\}$
    - Note: if $X_{t} = n$, algorithm halts with satisfying assignment $a^{*}$
    - Key point: suppose $a_{t}$ not a satisfying assignment and algorithm picks unsatisfied clause with variables $x_{i},x_{j}$
    - Note: since $a^{*}$ is satisfying, it makes a different assignment than $x_{i}$ or $x_{j}$ (or both)
    - Consequence of algorithm's random variable flip
        - (1) If $a_{t}$ and $a^{*}$ differ on both $x_{i}$ and $x_{j}$, then $X_{t+1} = X_{t} + 1$ (100% probability)
        - (2) If $a_{t}$ and $a^{*}$ differ on exactly one of $x_{i}$ and $x_{j}$, then $X_{t+1} = X_{t+1}$ (50% probability),  $X_{t-1}$ (50% probability)
    - Probability that a single iteration of the outer forloop finds a satisfying assignment is $\ge Pr[T_{n} \le 2n^{2}] \ge 1/2$
    - Thus, $Pr[$algorithm fails$] \le Pr[$all $log_{2}n$ independent trials fail]$ \le (\dfrac{1}{2})^{log_{2}n} = \dfrac{1}{n}$
    
## Wider world of algorithms

### Stable matching
- Consider two node sets $U$ and $V$ ("men" and "women")
- For simplicity: assumg $|U|=|V|=n$
- Each node has a ranked order of the nodes on the other side (different for different nodes)
- Ex. hospotals & residents, colleges & applicants
- Stable matching: a perfect matching (matches each node of $U$ to a different node of $V$) such that if $u \in U$ and $v \in V$ are not matched, then either $u$ likes its mate $v^{'}$ better than $v$ or $v$ likes its mate $u^{'}$ better then $u$

### Gale-Shapley proposal algorithm

- While there is an unsttached man $u$
    - $u$ proposes to the top woman $v$ on his preference list who hasn't rejected him yet
    - Each woman entertains only the best proposal received so far (invariant: current engagements = a matching)
- Theorem: terminates with a stable matching after $\le n^{2}$ iteration (in particular, a stable matching aways exist!)
- (1) Each man makes $\le n$ proposals => $\le n^{2}$ iterations
- (2) Terminates with a perfect matching, why? if not, some men rejected by all women
    - All $n$ women engaged at conclusion of algorithm
    - All $n$ men engaged at end, as well (contradiction)
- (3) Terminates with a stable matching, why? consider some $u,v$ not matched to each other
    - Case #1: $u$ never proposed to $v$
        - $u$ matched to someone he prefers to $v$
    - Case #2: $u$ proposed to $v$
        - $v$ got a better offer, ends up matched to someone she prefers to $u$
        
### Bipartite matching

- Input: bipartite graph $G = (U,V,E)$ (each $e \in E$ has one endpoint in each of $U,V$)
- Goal: compute a matching $M \in E$ (pairwise disjoint edges) of maximum size
- Fact: there is a straightforward reduction from this problem to the maximum flow problem

### Maximum flow problem

- Input: directed graph $G = (V,E)$
    - Source vertex $s$, sink vertex $t$
    - Each edge $e$ has capacity $u_{e}$
- Goal: compute the $s-t$ "flow" that sends as much flow as possbile
- Fact: solution in ploynomial time (via non-trivial greedy algorithms based on "augmenting path")

In [ ]:
import math
import random
from itertools import chai

In [ ]:
def open_graph(file_path):
    """
    Imports a file and stored data into a list of tuples

    Args:
    file_path (string) -- location of file

    Returns:
    tuple_data (tuple) -- list of clauses, set of clauses, integer representing number of clauses
    """

    graph = []
    rep = set()
    num_nodex = 0

    with open(file_path, 'r') as line:
        array = line.read().split("\n")
        num_nodes = int(array[0])
        del array[0]

        for subarray in array:
            x1 = int(subarray.split(" ")[0])
            x2 = int(subarray.split(" ")[1])

            graph.append((x1, x2))
            rep.add(x1)
            rep.add(x2)

    return (graph, rep, num_nodes)

In [ ]:
def get_values_to_delete(rep):
    """
    Prepare reduction where there is only 1 representation (not 2) of a variable
    
    Args:
    rep (set) -- unique set of clauses
    
    Returns:
    values_to_delete (set) -- clauses that can be deleted
    """

    values_to_delete = set()

    for i in rep:
        if -i not in rep:
            values_to_delete.add(i)

    return values_to_delete

In [ ]:
def update_rep(graph):
    """
    Acquire unique numbers that exist in the graph
    
    Args:
    graph (list) -- clauses
    
    Returns:
    rep (set) - unique set of clauses
    """

    rep = set()

    for item in graph:
        rep.add(item[0])
        rep.add(item[1])

    return rep

In [ ]:
def reduce_graph(graph, values_to_delete):
    """
    Apply reduction where there is only 1 representation (not 2) of a variable
    
    Args:
    graph (list) -- clauses
    values_to_delete (set) -- clauses that can be deleted
    
    Returns:
    new_graph (list) -- updated clauses after reduction has been applied
    """

    new_graph = []

    for item in graph:
        if item[0] not in values_to_delete and item[1] not in values_to_delete:
            new_graph.append(item)

    return new_graph

In [ ]:
def get_false_false_pair(graph, clause_table):
    """
    Find pair of caluse where both are set to False
    
    Args:
    graph (list) -- clauses
    clause_table (dictionary) -- clauses randomly set to either True or False 
    
    Returns:
    false_false_pair (list) -- pair of caluse where both are set to False
    """
    
    false_false_pair = []

    for item in graph:
        if clause_table[item[0]] == False and clause_table[item[1]] == False:
            false_false_pair.append(item)

    return false_false_pair

In [ ]:
def papadimitriou(graph, rep, clause_table):
    """
    Implement Papadimitriou algorithm for 2-SAT problem
    
    Args:
    graph (list) -- clauses
    rep (set) -- unique set of clauses
    clause_table (dictionary) -- clauses randomly set to either True or False 
    
    Returns:
    1 -- if saisfiable
    0 -- if non-saisfiable
    """
    
    for i in range(0, int(math.log(int(len(rep)/2), 2))): # repeat $log_{2}n$ times
        for j in range(0, 2 * (int(len(rep)/2) ** 2)): # repeat $2n^{2}$ times
            false_false_pair = get_false_false_pair(graph, clause_table) # find unsatisfiable clauses
            if len(false_false_pair) == 0:
                return 1
            else:
                item = random.choice(false_false_pair)
                value = abs(random.choice(item)) # pick a value to flip
                if clause_table[value] == True:
                    clause_table[value] = False
                    clause_table[-value] = True
                elif clause_table[value] == False:
                    clause_table[value] = True
                    clause_table[-value] = False

    return 0

In [ ]:
def compute_SAT(file_path):
    """
    Solve for 2-SAT problem
    
    Args:
    file_path (string) -- location of file
    
    Returns:
    1 -- if saisfiable
    0 -- if non-saisfiable
    """
    
    tuple_obj = open_graph(file_path)

    graph = tuple_obj[0]
    rep = tuple_obj[1]
    num_nodes = tuple_obj[2]

    print("------------")
    initial_while_loop = True
    values_to_delete = []
    while len(values_to_delete) > 0 or initial_while_loop == True:
        initial_while_loop = False
        values_to_delete = get_values_to_delete(rep)
        graph = reduce_graph(graph, values_to_delete)
        rep = update_rep(graph)
        
    print("len(graph) after final reduce: " + str(len(graph)))

    # Initial assignment of clauses
    clause_table = {}
    for item in rep:
        clause_table[item] = random.choice([True, False])
        clause_table[-item] = not clause_table[item]

    return papadimitriou(graph, rep, clause_table)

In [ ]:
assert(compute_SAT("data/2sat1.txt") == 1)
assert(compute_SAT("data/2sat2.txt") == 0)
assert(compute_SAT("data/2sat3.txt") == 1)
assert(compute_SAT("data/2sat4.txt") == 1)
assert(compute_SAT("data/2sat5.txt") == 0)
assert(compute_SAT("data/2sat6.txt") == 0)